# Transcription of the audio data and evaluation of its quality

This notebook aims to transcribe the audio data obtained from Egoli Media using the Google Spech-to-Text API. It also intends to evaluate the quality of the textual data obtained by the transcription.

In [ ]:
#Importing essential modules for the transcription
import pandas as pd 
import numpy as np 
import speech_recognition as sr
import os 
from pydub import AudioSegment
from pydub.silence import split_on_silence

In [ ]:
filename = "short.wav"
r = sr.Recognizer()

#open the file
with sr.AudioFile(filename) as source:
    #listen for the data
    audio_data = r.record(source, duration=300)
    audio_data = r.record(source, duration=29)

    #recognize (convert from speech to text)
    text = r.recognize_google(audio_data, language='en-UK', show_all=True)
    print(text)




In [ ]:
#Create a speech recognition object
r = sr.Recognizer()

#Function that splits the audio file into chunks and applies speech recognition
def get_large_audio_transcription(path):
    """
    Splitting the large audio file into chunks
    and apply speech recognition on each of these chunks
    """
    #Open the audio file using pydub
    sound = AudioSegment.from_wav(path)  
    #Split audio sound where silence is 700 miliseconds or more and get chunks
    chunks = split_on_silence(sound,
        min_silence_len = 300,
        silence_thresh = -2,
        keep_silence=500,
    )
    folder_name = "audio-chunks"
    #Create a directory to store the audio chunks
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)
    whole_text = ""
    print("chunks created")
    #Process each chunk 
    for i, audio_chunk in enumerate(chunks, start=1):
        chunk_filename = os.path.join(folder_name, f"chunk{i}.wav")
        audio_chunk.export(chunk_filename, format="wav")
        #Recognize the chunk
        with sr.AudioFile(chunk_filename) as source:
            audio_listened = r.record(source)
            #Try converting it to text
            try:
                text = r.recognize_google(audio_listened)
            except sr.UnknownValueError as e:
                print("Error:", str(e))
            except sr.RequestError:
                print("Error Request")
            else:
                text = f"{text.capitalize()}. "
                print(chunk_filename, ":", text)
                whole_text += text
    return whole_text

path = "short.wav"
print("\nFull text:", get_large_audio_transcription(path))

In [ ]:
#Opening the transcript of the Premier League audio if the previous step is not working because API limitations 
import json
with open('transcript_premierleague.json') as json_file:
    data = json.load(json_file)

whole_text = ""
for comment in data["results"]:
    whole_text += comment["alternatives"][0]["transcript"]
print(whole_text)

In [ ]:
#Opening the transcript of the Paralympic audio if the previous step is not working because API limitations 
with open('transcript_paralympic.txt') as f:
    lines = f.readlines()
whole_text = " ".join(lines).replace('"', "")
print(whole_text)

 We can easily notice that these transcriptions look very inaccurate and unreliable. It would be very difficult to build a high-performing event recognition and classification model with this data. Therefore, we will make use of the textual data obtained from live score websites to pursue our study. 